In [1]:
import boto3 

my_session = boto3.session.Session()
my_region = my_session.region_name
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]
algorithm_name = "facenet"

In [2]:
#remove ipython from requirement.txt
!./build_and_push.sh

230755935769
us-west-2
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
base_img:763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04
1.7.1-gpu-py36-cu110-ubuntu18.04: Pulling from pytorch-training

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
2f862619: Pulling fs layer 
278deddf: Pulling fs layer 
80049843: Pulling fs layer 
556b2329: Pulling fs layer 
a0c97a55: 

7bfc3338: Download complete  B/896.6MB32ADownloading  1.044GB/1.134GBDownloading  3.631kB/56.85kB0564d8a: Extracting  183.3MB/896.6MBB

Digest: sha256:cde54f998810f575e7e05453c784a1bf51b0afbc4c45f4ffe5b76c5e6f1d5e83[2K
Status: Downloaded newer image for 763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04
763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.7.1-gpu-py36-cu110-ubuntu18.04
Sending build context to Docker daemon  627.1MB
Step 1/8 : ARG BASE_IMG=${BASE_IMG}
Step 2/8 : FROM ${BASE_IMG}
 ---> 802fab1e085e
Step 3/8 : RUN git clone https://github.com/catwhiskers/facenet-pytorch.git
 ---> Running in 01ed6d0c46e7
Cloning into 'facenet-pytorch'...
Removing intermediate container 01ed6d0c46e7
 ---> 6d682bf1ed92
Step 4/8 : RUN cp changehostname.c /facenet-pytorch/
 ---> Running in 37e8e3e504aa
Removing intermediate container 37e8e3e504aa
 ---> 1ac209ca3c7f
Step 5/8 : RUN cd /facenet-pytorch && python setup.py install && pip install tensorboard==1.15
 ---> Running in 4b1e85151d2c
running install
running bdist_egg
running egg_info
creating facenet_pytorch.egg-info


Removing intermediate container 4b1e85151d2c
 ---> d81558a8a799
Step 6/8 : WORKDIR /facenet-pytorch
 ---> Running in 94567d7983cf
Removing intermediate container 94567d7983cf
 ---> 64265e26fbb1
Step 7/8 : ENV PATH="/facenet-pytorch:${PATH}"
 ---> Running in 1c1ad26274c6
Removing intermediate container 1c1ad26274c6
 ---> 687800505257
Step 8/8 : ENTRYPOINT ["python", "train.py"]
 ---> Running in 086db95bf672
Removing intermediate container 086db95bf672
 ---> ab247158a3c7
Successfully built ab247158a3c7
Successfully tagged facenet:latest
The push refers to repository [230755935769.dkr.ecr.us-west-2.amazonaws.com/facenet]

51a1aeb0: Preparing 
f859bb86: Preparing 
4bb20afd: Preparing 
9d32cd55: Preparing 
f1a6dc86: Preparing 
c65362d8: Preparing 
05a99e86: Preparing 
b53128e2: Preparing 
1b3406c9: Preparing 
31923288: Preparing 
e01e12be: Preparing 
bdcc3db4: Preparing 
ff09a22b: Preparing 
4502f184: Preparing 
f55816b2: Preparing 
a7715a11: Preparing 
00e53fc0: Preparing 
bfe29acb: Prepar

In [6]:
!cat Dockerfile

ARG BASE_IMG=${BASE_IMG}
FROM ${BASE_IMG} 
RUN git clone https://github.com/catwhiskers/facenet-pytorch.git
RUN cp changehostname.c /facenet-pytorch/
RUN cd /facenet-pytorch && python setup.py install && pip install tensorboard==1.15  
WORKDIR /facenet-pytorch
ENV PATH="/facenet-pytorch:${PATH}"
ENTRYPOINT ["python", "train.py"]


In [7]:
!cat build_and_push.sh

#!/bin/bash

# The name of our algorithm
algorithm_name=facenet

#cd container
# get information - account and region, required by ECR https://aws.amazon.com/ecr/
account=$(aws sts get-caller-identity --query Account --output text)
echo $account
# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}
echo $region


# derive fullname of docker image 
fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Get the login command from ECR in order to pull down the SageMaker TensorFlow image
$(aws ecr get-login 

In [8]:
image_uri=f"{account_id}.dkr.ecr.{my_region}.amazonaws.com/{algorithm_name}:latest"
image_uri

'230755935769.dkr.ecr.us-west-2.amazonaws.com/facenet:latest'

In [15]:
import sagemaker 
sess = sagemaker.session.Session()
bucket = sess.default_bucket()

In [16]:
import sagemaker 
session = sagemaker.session.Session()
bucket = session.default_bucket()

In [17]:
from sagemaker import get_execution_role

role = get_execution_role()
print(role)

s3_path = f"s3://{bucket}/data/faces"
s3_path

arn:aws:iam::230755935769:role/SageMakerExecutionRoleMLOps


's3://sagemaker-us-west-2-230755935769/data/faces'

In [19]:
# s3://facenet-demo-yianc 
# input.zip
!mkdir input 
!mkdir input/data
!mv examples/lfw input/data/faces


!aws s3 cp --recursive ./input/data/faces/ $s3_path

mkdir: cannot create directory ‘input’: File exists
mkdir: cannot create directory ‘input/data’: File exists
upload: input/data/faces/Aaron_Eckhart/Aaron_Eckhart_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Aaron_Eckhart/Aaron_Eckhart_0001.jpg
upload: input/data/faces/AJ_Lamas/AJ_Lamas_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/AJ_Lamas/AJ_Lamas_0001.jpg
upload: input/data/faces/AJ_Cook/AJ_Cook_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/AJ_Cook/AJ_Cook_0001.jpg
upload: input/data/faces/Aaron_Patterson/Aaron_Patterson_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Aaron_Patterson/Aaron_Patterson_0001.jpg
upload: input/data/faces/Aaron_Peirsol/Aaron_Peirsol_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Aaron_Peirsol/Aaron_Peirsol_0002.jpg
upload: input/data/faces/Aaron_Sorkin/Aaron_Sorkin_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Aaron_Sorkin/Aaron_Sorkin_0002.jpg
upload: input/data/faces/

upload: input/data/faces/Abel_Pacheco/Abel_Pacheco_0003.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Abel_Pacheco/Abel_Pacheco_0003.jpg
upload: input/data/faces/Abel_Pacheco/Abel_Pacheco_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Abel_Pacheco/Abel_Pacheco_0002.jpg
upload: input/data/faces/Abdullah_Gul/Abdullah_Gul_0019.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Abdullah_Gul/Abdullah_Gul_0019.jpg
upload: input/data/faces/Abel_Pacheco/Abel_Pacheco_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Abel_Pacheco/Abel_Pacheco_0001.jpg
upload: input/data/faces/Abel_Pacheco/Abel_Pacheco_0004.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Abel_Pacheco/Abel_Pacheco_0004.jpg
upload: input/data/faces/Abid_Hamid_Mahmud_Al-Tikriti/Abid_Hamid_Mahmud_Al-Tikriti_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Abid_Hamid_Mahmud_Al-Tikriti/Abid_Hamid_Mahmud_Al-Tikriti_0001.jpg
upload: input/data/faces/Abid_Hamid_Mahmud_Al-Tikriti/

upload: input/data/faces/Adrien_Brody/Adrien_Brody_0008.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Adrien_Brody/Adrien_Brody_0008.jpg
upload: input/data/faces/Agnelo_Queiroz/Agnelo_Queiroz_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Agnelo_Queiroz/Agnelo_Queiroz_0001.jpg
upload: input/data/faces/Afton_Smith/Afton_Smith_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Afton_Smith/Afton_Smith_0001.jpg
upload: input/data/faces/Ahmad_Masood/Ahmad_Masood_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Ahmad_Masood/Ahmad_Masood_0002.jpg
upload: input/data/faces/Adrien_Brody/Adrien_Brody_0012.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Adrien_Brody/Adrien_Brody_0012.jpg
upload: input/data/faces/Ahmad_Masood/Ahmad_Masood_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Ahmad_Masood/Ahmad_Masood_0001.jpg
upload: input/data/faces/Ahmed_Ahmed/Ahmed_Ahmed_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Ahme

upload: input/data/faces/Al_Gore/Al_Gore_0008.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Al_Gore/Al_Gore_0008.jpg
upload: input/data/faces/Al_Leiter/Al_Leiter_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Al_Leiter/Al_Leiter_0001.jpg
upload: input/data/faces/Al_Pacino/Al_Pacino_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Al_Pacino/Al_Pacino_0001.jpg
upload: input/data/faces/Al_Pacino/Al_Pacino_0003.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Al_Pacino/Al_Pacino_0003.jpg
upload: input/data/faces/Al_Sharpton/Al_Sharpton_0004.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Al_Sharpton/Al_Sharpton_0004.jpg
upload: input/data/faces/Al_Sharpton/Al_Sharpton_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Al_Sharpton/Al_Sharpton_0002.jpg
upload: input/data/faces/Al_Sharpton/Al_Sharpton_0007.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Al_Sharpton/Al_Sharpton_0007.jpg
upload: input/data/faces/Al_Sharpton/Al

upload: input/data/faces/Alec_Baldwin/Alec_Baldwin_0004.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Alec_Baldwin/Alec_Baldwin_0004.jpg
upload: input/data/faces/Alec_Baldwin/Alec_Baldwin_0003.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Alec_Baldwin/Alec_Baldwin_0003.jpg
upload: input/data/faces/Alecos_Markides/Alecos_Markides_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Alecos_Markides/Alecos_Markides_0001.jpg
upload: input/data/faces/Alejandro_Atchugarry/Alejandro_Atchugarry_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Alejandro_Atchugarry/Alejandro_Atchugarry_0002.jpg
upload: input/data/faces/Aldo_Paredes/Aldo_Paredes_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Aldo_Paredes/Aldo_Paredes_0002.jpg
upload: input/data/faces/Alejandro_Avila/Alejandro_Avila_0003.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Alejandro_Avila/Alejandro_Avila_0003.jpg
upload: input/data/faces/Alejandro_Fernandez/Alejandro_Fernand

...

upload: input/data/faces/Wen_Jiabao/Wen_Jiabao_0012.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Wen_Jiabao/Wen_Jiabao_0012.jpg
upload: input/data/faces/Wen_Jiabao/Wen_Jiabao_0005.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Wen_Jiabao/Wen_Jiabao_0005.jpg
upload: input/data/faces/Wen_Jiabao/Wen_Jiabao_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Wen_Jiabao/Wen_Jiabao_0001.jpg
upload: input/data/faces/Wen_Jiabao/Wen_Jiabao_0011.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Wen_Jiabao/Wen_Jiabao_0011.jpg
upload: input/data/faces/Wayne_Newton/Wayne_Newton_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Wayne_Newton/Wayne_Newton_0001.jpg
upload: input/data/faces/Wei_Wu/Wei_Wu_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Wei_Wu/Wei_Wu_0001.jpg
upload: input/data/faces/Wen_Jiabao/Wen_Jiabao_0007.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Wen_Jiabao/Wen_Jiabao_0007.jpg
upload: input/data/faces/Wen_Jiabao

upload: input/data/faces/William_Hochul/William_Hochul_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/William_Hochul/William_Hochul_0002.jpg
upload: input/data/faces/William_Macy/William_Macy_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/William_Macy/William_Macy_0001.jpg
upload: input/data/faces/William_Harrison/William_Harrison_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/William_Harrison/William_Harrison_0001.jpg
upload: input/data/faces/William_Jackson/William_Jackson_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/William_Jackson/William_Jackson_0001.jpg
upload: input/data/faces/William_Macy/William_Macy_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/William_Macy/William_Macy_0002.jpg
upload: input/data/faces/William_Joppy/William_Joppy_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/William_Joppy/William_Joppy_0001.jpg
upload: input/data/faces/William_Hyde/William_Hyde_0001.jpg to s3://sagemaker-

upload: input/data/faces/Yasar_Yakis/Yasar_Yakis_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yasar_Yakis/Yasar_Yakis_0001.jpg
upload: input/data/faces/Yao_Ming/Yao_Ming_0008.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yao_Ming/Yao_Ming_0008.jpg
upload: input/data/faces/Yashwant_Sinha/Yashwant_Sinha_0003.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yashwant_Sinha/Yashwant_Sinha_0003.jpg
upload: input/data/faces/Yao_Ming/Yao_Ming_0006.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yao_Ming/Yao_Ming_0006.jpg
upload: input/data/faces/Yasar_Yakis/Yasar_Yakis_0004.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yasar_Yakis/Yasar_Yakis_0004.jpg
upload: input/data/faces/Yashwant_Sinha/Yashwant_Sinha_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yashwant_Sinha/Yashwant_Sinha_0002.jpg
upload: input/data/faces/Yashwant_Sinha/Yashwant_Sinha_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yashwant_Sinha/Yashwant_Si

upload: input/data/faces/Yory_Boy_Campas/Yory_Boy_Campas_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yory_Boy_Campas/Yory_Boy_Campas_0001.jpg
upload: input/data/faces/Yukiko_Okudo/Yukiko_Okudo_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yukiko_Okudo/Yukiko_Okudo_0001.jpg
upload: input/data/faces/Yu_Shyi-kun/Yu_Shyi-kun_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yu_Shyi-kun/Yu_Shyi-kun_0001.jpg
upload: input/data/faces/Yukiko_Okudo/Yukiko_Okudo_0002.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yukiko_Okudo/Yukiko_Okudo_0002.jpg
upload: input/data/faces/Yu_Shyi-kun/Yu_Shyi-kun_0004.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yu_Shyi-kun/Yu_Shyi-kun_0004.jpg
upload: input/data/faces/Yuri_Luzhkov/Yuri_Luzhkov_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yuri_Luzhkov/Yuri_Luzhkov_0001.jpg
upload: input/data/faces/Yu_Shyi-kun/Yu_Shyi-kun_0003.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Yu_S

upload: input/data/faces/Zulfiqar_Ahmed/Zulfiqar_Ahmed_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Zulfiqar_Ahmed/Zulfiqar_Ahmed_0001.jpg
upload: input/data/faces/Zumrati_Juma/Zumrati_Juma_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Zumrati_Juma/Zumrati_Juma_0001.jpg
upload: input/data/faces/Ziwang_Xu/Ziwang_Xu_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Ziwang_Xu/Ziwang_Xu_0001.jpg
upload: input/data/faces/Zurab_Tsereteli/Zurab_Tsereteli_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Zurab_Tsereteli/Zurab_Tsereteli_0001.jpg
upload: input/data/faces/Zydrunas_Ilgauskas/Zydrunas_Ilgauskas_0001.jpg to s3://sagemaker-us-west-2-230755935769/data/faces/Zydrunas_Ilgauskas/Zydrunas_Ilgauskas_0001.jpg


### can train in docker container manually 

* start container in interactive mode
```
docker run -it -v $PWD:/opt/ml --entrypoint '' $image_uri bash 
```
* train model based on README.md
```
python train.py --csv_path=/opt/ml/input/data/competition/meta_train.csv --data_dir=/opt/ml/input/data/competition/train --epochs=50 --val_split 0.1
```

In [20]:
from datetime import datetime
now = datetime.now()
timestamp = datetime.timestamp(now)
job_name = "facenet-{}".format(str(int(timestamp))) 
job_name

'facenet-1621843062'

In [21]:
estimator = sagemaker.estimator.Estimator( 
                        role=role,
                        image_uri=image_uri,
                        instance_count=1,
                        instance_type='ml.p3.8xlarge',
                        sagemaker_session=session,
                        volume_size=100, 
                        debugger_hook_config=False
                   )

In [ ]:
estimator.fit(inputs={"faces":s3_path}, job_name=job_name)

2021-05-24 07:58:02 Starting - Starting the training job...
2021-05-24 07:58:25 Starting - Launching requested ML instancesProfilerReport-1621843081: InProgress
.........
2021-05-24 07:59:45 Starting - Preparing the instances for training......
2021-05-24 08:00:45 Downloading - Downloading input data......
2021-05-24 08:01:57 Training - Downloading the training image..........................Running on device: cuda:0
[2021-05-24 08:06:12.219 ip-10-0-114-112.us-west-2.compute.internal:1 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2021-05-24 08:06:12.288 ip-10-0-114-112.us-west-2.compute.internal:1 INFO profiler_config_parser.py:102] User has disabled profiler.
/facenet-pytorch/models/utils/detect_face.py:210: UserWarning: This overload of nonzero is deprecated:
#011nonzero()
Consider using one of the following signatures instead:
#011nonzero(*, bool as_tuple) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:882.)
  mask_inds = mask.nonzero()
/facenet-pytorc

In [23]:
model_path = estimator.model_data

In [27]:
!aws s3 cp $model_path . 
!tar -xf model.tar.gz
!ls -al fine_tuned_model


download: s3://sagemaker-us-west-2-230755935769/facenet-1621843062/output/model.tar.gz to ./model.tar.gz
-rw-r--r-- 1 ec2-user ec2-user 106165275 May 24 08:23 fine_tuned_model
